#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2017


# Homework 2:  Link Analysis -- PageRank + SEO

### 100 points [5% of your final grade]

### Due: Tuesday, February 21, 2017 by 11:59pm

*Goals of this homework:* Explore real-world challenges of building a graph (in this case, from tweets), implement and test PageRank over this graph, and investigate factors that impact a page's rank on Google and Bing.

*Submission Instructions:* To submit your homework, rename this notebook as YOUR_UIN_hw2.ipynb. Submit this notebook via ecampus. Your notebook should be completely self-contained, with the results visible in the notebook. 

*Late submission policy:* For this homework, you may use up to three of your late days, meaning that no submissions will be accepted after Friday, February 24, 2017 at 11:59pm.

# Part 1: PageRank (70 points)

## A Twitter-Mentioning Graph

In this assignment, we're going to adapt the classic PageRank approach to allow us to find not the most authoritative web pages, but rather to find significant Twitter users. So, instead of viewing the world as web pages with hyperlinks (where pages = nodes, hyperlinks = edges), we're going to construct a graph of Twitter users and their mentions of other Twitter users (so user = node, mention of another user = edge). Over this Twitter-user graph, we can apply the PageRank approach to order the users. The main idea is that a user who is mentioned by other users is more "impactful". 

Here is a toy example. Suppose you are given the following four tweets:

* **userID**: diane, **text**: "@bob Howdy!"
* **userID**: charlie, **text**: "Welcome @bob and @alice!"
* **userID**: bob, **text**: "Hi @charlie and @alice!"
* **userID**: alice, **text**: "Howdy!"

There are four short tweets generated by four users. The @mentions between users form a directed graph with four nodes and five edges. E.g., the "diane" node has a directed edge to the "bob" node. Note that a retweet also contain the "@", so it should be counted as a mention as well.

You should build a graph by parsing the tweets in the file we provide called *pagerank.json*.

**Notes:**

* The edges are binary and directed. If Bob mentions Alice once, in 10 tweets, or 10 times in one tweet, there is an edge from Bob to Alice, but there is not an edge from Alice to Bob.
* If a user mentions herself, ignore it.
* Correctly parsing @mentions in a tweet is error-prone. Use the entities field.
* Later you will need to implement the PageRank algorithm on the graph you build here.


In [70]:
# Here define your function for building the graph by parsing the input file of tweets
# Insert as many cells as you want

import json
data=[] #storing the tweets in data
with open('pagerank.json', 'r') as f:
    for line in f:
        data.append(json.loads(line))


In [71]:
#First I am creating a dictionary with key as id of person tweeting and value as address of a linked list whose nodes are id of people about whom key has tweeted
class node(object):
    def __init__(self,a,p = None):
        self.value = a
        self.pointer = p
    def get_next(self):
        return self.pointer
    def get_value(self):
        return self.value
    def set_next(self,p):
        self.pointer = p
    def set_value(self,a):
        self.value = a

class build_list(object):
    def __init__(self,r=None):
        self.root = r
        self.size = 0
    def get_size(self):     #returns the size of linked list
        return self.size
    def add_link(self,d):   #this inserts the id of person about whom someone is tweeting
        new_node = node(d,self.root)
        self.root = new_node
        self.size += 1
    def find(self,d):     #finds if the id of a person is already present in the linkedlist
        this_node = self.root
        while (this_node != None):
            if this_node.get_value() == d:
                return True
            else:
                this_node = this_node.get_next()
        return False

In [72]:
id_to_screenname={}    #this is a dictionary with key as screen name and value as id so even if a person has more than one usernames the corresponding id will be the same
for line in data:
    id_user = line['user']['id']
    screenname_user = line['user']['screen_name']
    id_to_screenname[screenname_user] = id_user
    user_entities = line['entities']['user_mentions']#[0]['screen_name']
    for j in range(0,(len(user_entities))):    #extracting the names and id of persons being tweeted
        extract_user_names = user_entities[j]['screen_name']
        extracted_userid = user_entities[j]['id']
        id_to_screenname[extract_user_names] = extracted_userid


In [73]:
#building the linked list in this cell. id_links is the dictionary whose key are id and values are addresses of linked list
id_links={}
user_entities=[]
i=0
for line in data:
    id_user = line['user']['id']
    screenname_user = line['user']['screen_name']
    user_entities = line['entities']['user_mentions']
    if id_user not in id_links:  #Checking if the person who posted the tweet is in our dict or noi
        new_id = build_list()
        id_links[id_user] = new_id
    for j in range(0,(len(user_entities))): #extracting people mentioned in tweets using entities
            extract_user_names = user_entities[j]['screen_name']
            extracted_username_id = id_to_screenname[extract_user_names]
            if extracted_username_id in id_links:   #checking if the extracted person has a node or not
                if (id_links[id_user].find(extracted_username_id) == False): #This is used to check that a person is added only once even if he is mentioned more than once by a person
                    if (extracted_username_id != id_user):  #to check the person is not tweeting about himself
                        id_links[id_user].add_link(extracted_username_id)
                        i = i+1
            else:
                new_id1 = build_list()
                id_links[extracted_username_id] = new_id1
                if (id_links[id_user].find(extracted_username_id) == False): #This is used to check that a person is added only once even if he is mentioned more than once by a person
                    if (extracted_username_id != id_user):   #to check the person is not tweeting about himself
                        id_links[id_user].add_link(extracted_username_id)  
                        i = i+1
        


In [74]:
# Call your function to print out the size of the graph, i.e., the number of nodes and edges
# How you maintain the graph is totaly up to you
# However, if you encounter any memory issues, we recommend you write the graph into a file, and load it later.
jj = 0
for idd in id_links:
    jj = jj + id_links[idd].get_size()

        
print "Number of nodes in graph is ", len(id_links), "Number of edges in graph is", jj


Number of nodes in graph is  16430 Number of edges in graph is 24257


We will not check the correctness of your graph. However, this will affect the PageRank results later.

## PageRank Implementation

Your program will return the top 10 users with highest PageRank scores. The **output** should be like:

* user1 - score1
* user2 - score2
* ...
* user10 - score10

You should follow these **rules**:

* Assume all nodes start out with equal probability.
* The probability of the random surfer teleporting is 0.1 (that is, the damping factor is 0.9).
* If a user is never mentioned and does not mention anyone, their PageRank scores should be zero. Do not include the user in the calculation.
* It is up to you to decide when to terminate the PageRank calculation.
* There are PageRank implementations out there on the web. Remember, your code should be **your own**.


**Hints**:
* If you're using the matrix style approach, you should use [numpy.matrix](https://docs.scipy.org/doc/numpy/reference/generated/numpy.matrix.html).
* Scipy is built on top of Numpy and has support for sparse matrices. You most likely will not need to use Scipy unless you'd like to try out their sparse matrices.
* If you choose to use Numpy (and Scipy), please make sure your Anaconda environment include their latest versions (Numpy 1.12.0; Scipy 0.18.1).
* Test your parsing and PageRank calculations using a handful of tweets, before moving on to the entire file we provide.
* We will evaluate the user ranks you provide as well as the quality of your code. So make sure that your code is clear and readable.

What is the termination condition in your PageRank implementation? Describe it below:

*ADD YOUR INPUT HERE*
I am comparing the top 10 scores between two successive iterations. if the difference between their scores is less than 0.000001 then the loop stops or else if it completes 100 iterations before reaching the previous said condition then also loop stops

In [75]:
class node_pr(object):      #This node contains the user id and the number of peoples names tweeted by that id which is given by outlinks
    def __init__(self,a,ol,p = None):
        self.value = a     #id of people who tweeted about this person
        self.outlinks = ol # No. of people tweeted by this node
        self.pointer = p
    def get_next(self):
        return self.pointer
    def get_value(self):
        return self.value
    def get_outlinks(self):
        return self.outlinks
    def set_next(self,p):
        self.pointer = p
    def set_value(self,a):
        self.value = a
    def set_outlinks(self,ol):
        self.outlinks = ol

class build_list_pr(object):
    def __init__(self,r=None):
        self.root = r
        self.size = 0
    def get_size(self):     #returns the size of linked list
        return self.size
    def add_link(self,d,ol):   #this inserts the id of person about whom someone is tweeting
        new_node = node_pr(d,ol,self.root)
        self.root = new_node
        self.size += 1
    def find(self,d):     #finds if the id of a person is already present in the linkedlist
        this_node = self.root
        while (this_node != None):
            if this_node.get_value() == d:
                return True
            else:
                this_node = this_node.get_next()
        return False

In [80]:
#id_to_prcal contains a new dictionary with key being users id and value containing address of lists. these lists contain the userid of persons who tweeted about this person and the no.of people he tweeted about 
def Pageranker(id_links):
    id_to_prcal={}     
    convergence1={}   #this dictionary stores the scores of a iteration. if covergence2 holds scores of 8 iteration then convergence1 has scores of 7 iteration
    convergence2={}   #this dictionary stores the scores of a iteration
    for idd in id_links:
        if idd not in id_to_prcal:
            new_id = build_list_pr()
            id_to_prcal[idd] = new_id
        now = id_links[idd].root
        while (now != None):
            if now.get_value() not in id_to_prcal:
                new_id1 = build_list_pr()
                id_to_prcal[now.get_value()] = new_id1
                id_to_prcal[now.get_value()].add_link(idd,id_links[idd].get_size())
            else:
                id_to_prcal[now.get_value()].add_link(idd,id_links[idd].get_size())
            now = now.get_next()
    id_to_pr={}  #I am creating a dictionary to store the page rank values of previous iterations
    for iddd in id_to_prcal:
            id_to_pr[iddd] = id_to_prcal[iddd]

    id_to_pr2={}  #this stores the pr values after calculation in that iteration
    for iddd in id_to_prcal:
            id_to_pr2[iddd] = id_to_prcal[iddd]
    for idd in id_to_prcal:
        id_to_pr[idd] = 1.0/len(id_to_pr)  #initial pagerank values of all users is equal to 1/16430
        convergence1[idd] = id_to_pr[idd]

    N = len(id_to_pr2)   #No.of nodes
    d = 0.9              #damping factor
    constant = (0.1)/N   #(1-d)/N
    flag = 1
    for i in range(0,100):
        for idd in id_to_prcal:
            sum = 0.0
            now = id_to_prcal[idd].root
            while (now != None):   #PR of a user is the sum of PR's of people who tweeted about this user divided by the no.of outlinks of that particular user
                sum = sum + float(id_to_pr[now.get_value()])/float(now.get_outlinks())
                now = now.get_next()
            sum = (sum * d) + constant
            id_to_pr2[idd] = sum
        for iddd in id_to_pr2:
            id_to_pr[iddd] = id_to_pr2[iddd]
        iii=0
        for entry in sorted(id_to_pr.items(), key = lambda x: x[1], reverse=True):  #used to sort the dictionary in descending order according to values
            convergence2[entry[0]] = entry[1]
            if ((convergence2[entry[0]] - convergence1[entry[0]]) < 0.000001):  #difference between successive iterations 
                flag = flag * 1
            else:
                flag = 0
            convergence1[entry[0]] = convergence2[entry[0]]
            iii = iii + 1
            if iii == 10:
                break;
        if (flag == 1):
            break
        else:
            flag = 1
            iii = 0
    #print "iii is ", iii, "i is ",i
    sum_square=0
    for idd in id_to_pr:
        sum_square = sum_square + id_to_pr[idd]*id_to_pr[idd]
    convergence[idd] = id_to_pr[idd]   
    sqrt_sum_square= sum_square ** (0.5)
    #print "sum_square is ", sum_square," square root is ", sqrt_sum_square
    for idd in id_to_pr:
        id_to_pr[idd] = id_to_pr[idd]/sqrt_sum_square
    print "Top 10 popular users are and their scores are "
    i=0  
    for entry in sorted(id_to_pr.items(), key = lambda x: x[1], reverse=True):
        #print entry#, id_to_pr[entry]
        print (i+1),": ",entry[0],"     ",entry[1]
        i = i+1
        if i==10:
            break
    return



In [81]:
# Now let's call your function on the graph you've built. Output the results.


Pageranker(id_links)

Top 10 popular users are and their scores are 
1 :  158314798       0.26991862085
2 :  181561712       0.22567202354
3 :  209708391       0.211007923707
4 :  72064417       0.192328101174
5 :  105119490       0.176778657467
6 :  14268057       0.162529194502
7 :  379961664       0.15369485501
8 :  391037985       0.147867588768
9 :  153074065       0.147751152659
10 :  313525912       0.14224206826


# Part 2: Search Engine Optimization (30 + 5 points)

For this part, your goal is to put on your "[search engine optimization](https://en.wikipedia.org/wiki/Search_engine_optimization)" hat. Your job is to create a webpage that scores highest for the query: **awcv9kjlh scwrlkjf4e** --- two terms, lower case, no quote. As of today (Feb 7, 2017), there are no hits for this query on either Google or Bing. Based on our discussions of search engine ranking algorithms, you know that several factors may impact a page's rank. Your goal is to use this knowledge to promote your own page to the top of the list.

What we're doing here is a form of [SEO contest](https://en.wikipedia.org/wiki/SEO_contest). While you have great latitude in how you approach this problem, you are not allowed to engage in any unethical or illegal behavior. Please read the discussion of "white hat" versus "black hat" SEO over at [Wikipedia](https://en.wikipedia.org/wiki/Search_engine_optimization).


**Rules of the game:**

* Somewhere in the page (possibly in the non-viewable source html) you must include your name or some other way for us to identify you (e.g., your NetID, but not the UIN!).
* Your target page may only be a TAMU student page, a page on your own webserver, a page on a standard blog platform (e.g., wordpress), or some other primarily user-controlled page
* Your target page CAN NOT be a twitter account, a facebook page, a Yahoo Answers or similar page
* No wikipedia vandalism
* No yahoo/wiki answers questions
* No comment spamming of blogs
* If you have concerns/questions/clarifications, please post on Piazza and we will discuss

For your homework turnin for this part, you should provide us the URL of your target page and a brief discussion (2-4 paragraphs) of the strategies you are using. We will issue the query and check the rankings at some undetermined time in the next couple of weeks. You might guess that major search engines take some time to discover and integrate new pages: if I were you, I'd get a target page up immediately.

**Grading:**

* 5 points for providing a valid URL
* 20 points for a well-reasoned discussion of your strategy
* 5 points for your page appearing in the search results by Google or Bing (no matter how is the ranking)

** Bonus: **
* 1 point for your page appearing in the top-20 on Google or Bing
* 1 more point for your page appearing in the top-10 on Google or Bing
* 1 more point for your page appearing in the top-5 on Google or Bing
* 2 more points for your page being ranked first by Google or Bing. And, a vigorous announcement in class, and a high-five for having the top result!

What's the URL of your page?

*ADD YOUR INPUT HERE*
http://people.tamu.edu/~karthik.273/

What's your strategy? (2-4 paragraphs)

*ADD YOUR INPUT HERE*
My strategy is first I used tamu.edu domain. As it is a well known domain it gives higher priority. After that I searched about the top 100 most searched words on google, added hyperlinks to their google searches and kept all of these on my webpage. I was thinking as these are mostly searched words, the chance of my page getting also selected because of the presence of word will increase. After that, I created a question in Quora asking how to increase my rank and giving a link to my webpage to show what I did on so I get a backlink from Quora